In [ ]:
from helper import get_openai_api_key


In [2]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=openai_api_key
)

In [3]:
model = "gpt-4o-mini"

In [4]:
system_prompt = "You are a chatbot."

In [5]:
import letta

In [12]:
# Make the completion request with the tool usage
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window 
        {"role": "system", "content": system_prompt}, 
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"}, 
    ]
)
chat_completion.choices[0].message.content

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************Ji0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [13]:
agent_memory = {"human": "Name: Bob"}
system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

In [14]:
agent_memory = {"human": "Name: Bob"}
system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

In [15]:
import json


chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt + "[MEMORY]\n" + \
         json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "What is my name?"},
    ],
)
chat_completion.choices[0].message.content

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************Ji0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
agent_memory = {"human": "", "agent": ""}

def core_memory_save(section: str, memory: str): 
    agent_memory[section] += '\n' 
    agent_memory[section] += memory 

In [ ]:
agent_memory

{'human': '', 'agent': ''}

In [ ]:
core_memory_save("human", "The human's name is Charles")

In [ ]:
agent_memory

{'human': "\nThe human's name is Charles", 'agent': ''}

In [ ]:
# tool description 
core_memory_save_description = "Save important information about you," \
+ "the agent or the human you are chatting with."

# arguments into the tool (generated by the LLM)
# defines what the agent must generate to input into the tool 
core_memory_save_properties = \
{
    # arg 1: section of memory to edit
    "section": {
        "type": "string",
        "enum": ["human", "agent"],
        "description": "Must be either 'human' " \
        + "(to save information about the human) or 'agent'" \
        + "(to save information about yourself)",            
    },
    # arg 2: memory to save
    "memory": {
        "type": "string",
        "description": "Memory to save in the section",
    },
}

# tool schema (passed to OpenAI)
core_memory_save_metadata = \
    {
        "type": "function",
        "function": {
            "name": "core_memory_save",
            "description": core_memory_save_description,
            "parameters": {
                "type": "object",
                "properties": core_memory_save_properties,
                "required": ["section", "memory"],
            },
        }
    }

In [ ]:
agent_memory = {"human": ""}
system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "My name is Bob"},
    ],
    # tool schemas 
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oedjruwiMaBRZR14TRUwBazA', function=Function(arguments='{"section":"human","memory":"The human\'s name is Bob."}', name='core_memory_save'), type='function')]))

In [ ]:
arguments = json.loads(response.message.tool_calls[0].function.arguments)
arguments

{'section': 'human', 'memory': "The human's name is Bob."}

In [ ]:
# run the function with the specified arguments 
core_memory_save(**arguments)

In [ ]:
agent_memory

{'human': "\nThe human's name is Bob."}

In [ ]:
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "what is my name"},
    ],
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response.message

ChatCompletionMessage(content='Your name is Bob.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [ ]:
agent_memory = {"human": ""}

In [ ]:
system_prompt_os = system_prompt \
+ "\n. You must either call a tool (core_memory_save) or" \
+ "write a response to the user. " \
+ "Do not take the same actions multiple times!" \
+ "When you learn new information, make sure to always" \
+ "call the core_memory_save tool." 

In [ ]:
def agent_step(user_message): 
    
    # prefix messages with system prompt and memory
    messages = [
        # system prompt 
        {"role": "system", "content": system_prompt_os}, 
        # memory
        {
            "role": "system", 
            "content": "[MEMORY]\n" + json.dumps(agent_memory)
        },
    ]    

    # append the most recent message
    messages.append({"role": "user", "content": user_message})
    
    # agentic loop 
    while True: 
        chat_completion = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=[core_memory_save_metadata]
        )
        response = chat_completion.choices[0]

        # update the messages with the agent's response 
        messages.append(response.message)
        
        # if NOT calling a tool (responding to the user), return 
        if not response.message.tool_calls: 
            return response.message.content

        # if calling a tool, execute the tool
        else: 
            print("TOOL CALL:", response.message.tool_calls[0].function)
            
            # parse the arguments from the LLM function call
            arguments = json.loads(
                response.message.tool_calls[0].function.arguments
            )

            # run the function with the specified arguments
            core_memory_save(**arguments)

            # add the tool call response to the message history 
            messages.append({
                "role": "tool", 
                "tool_call_id": response.message.tool_calls[0].id, 
                "name": "core_memory_save", 
                "content": f"Updated memory: {json.dumps(agent_memory)}"
            })

In [ ]:
agent_step("my name is malvika.")

TOOL CALL: Function(arguments='{"section":"human","memory":"The user\'s name is Malvika."}', name='core_memory_save')


'Nice to meet you, Malvika! How can I assist you today?'

In [ ]:
agent_memory = {"human": ""}

In [ ]:
system_prompt_os = system_prompt \
+ "\n. You must either call a tool (core_memory_save) or" \
+ "write a response to the user. " \
+ "Do not take the same actions multiple times!" \
+ "When you learn new information, make sure to always" \
+ "call the core_memory_save tool." 

In [ ]:
import json
import logging

# Disable httpx logging
logging.basicConfig(level=logging.CRITICAL)  # Disable lower-level logs, like info/debug
logging.getLogger("httpx").setLevel(logging.CRITICAL)

# Initialize user and agent names (can be set dynamically)
user_name = "John Doe"  # Default name
agent_name = "Letta"  # Agent's name

# Sample memory structure that stores context (name, preferences, answers, tasks, and conversations)
agent_memory = {
    'user_name': user_name,
    'agent_name': agent_name,
    'previous_answers': {},
    'tasks': {},  # Store tasks in this section
    'conversations': []  # Store only user and agent's responses in this section
}

# Define system prompts and memory with instruction to always call core_memory_save
def get_system_prompt():
    return """
    You are {agent_name}, a friendly assistant who is helping {user_name}.
    Please use their name and the context in your responses.
    You must either call a tool (core_memory_save) or write a response to the user. 
    Do not take the same actions multiple times! 
    When you learn new information, make sure to always call the core_memory_save tool.
    """.format(agent_name=agent_name, user_name=agent_memory['user_name'])

# Function to save updated memory (simulating saving to a storage or database)
def core_memory_save(**kwargs):
    """
    Save the updated memory (for example, storing answers or preferences).
    Here we simulate updating the global agent_memory.
    """
    global agent_memory
    for key, value in kwargs.items():
        agent_memory[key] = value

# Main agent step function (user interaction)
def agent_step(user_message):
    """
    This function handles the conversation and updates memory based on user input.
    It simulates an agent that updates memory and calls tools as needed.
    """
    global user_name
    
    # Check if the user mentions their name and update memory
    if "my name is" in user_message.lower():
        new_name = user_message.lower().replace("my name is", "").strip()
        agent_memory['user_name'] = new_name
        user_name = new_name  # Update the user_name variable
        core_memory_save(user_name=new_name)  # Update memory with the new name
        return f"Got it, {new_name}! I'll call you by that name from now on."

    # Store the user input and agent's response in the conversation history
    messages = [
        # system prompt
        {"role": "system", "content": get_system_prompt()},
        # memory (current state with only agent's conversations)
        {
            "role": "system", 
            "content": "[MEMORY]\n" + json.dumps({
                'user_name': agent_memory['user_name'],
                'agent_name': agent_memory['agent_name'],
                'conversations': agent_memory['conversations']
            }, indent=2)
        },
    ]    

    # Append the most recent message from the user
    messages.append({"role": "user", "content": user_message})

    # Agentic loop (conversation loop with tool invocation checks)
    while True:
        # Assuming `client.chat.completions.create` is set up and functional
        chat_completion = client.chat.completions.create(
            model="gpt-4",  # Or whichever model you're using
            messages=messages,
        )
        
        response = chat_completion.choices[0]

        # Store only user and agent's responses in the memory
        agent_memory['conversations'].append({
            "role": "user", 
            "content": user_message
        })
        agent_memory['conversations'].append({
            "role": "agent", 
            "content": response.message.content
        })

        # Add agent's response to the memory
        core_memory_save(conversations=agent_memory['conversations'])

        # Return the agent's response
        return response.message.content

# Example usage:
# Start the conversation by calling the agent_step function
user_input = input("You: ")
while user_input.lower() != "bye":
    response = agent_step(user_input)
    print(f"{agent_memory['agent_name']}: {response}")  # Print agent's name dynamically
    user_input = input("You: ")

print(f"{agent_memory['agent_name']}: Goodbye!")  # Print agent's name dynamically


Letta: Hello, John Doe! How can I assist you today?
Letta: Nice to meet you, Malviak. How can I assist you today?
Letta: Hello, Malviak. How can I assist you today?


KeyboardInterrupt: Interrupted by user

In [ ]:
import sqlite3
print(sqlite3.version)  # Should print the SQLite version


2.6.0


/var/folders/4g/v3_5f9ss3318x3sz6frh74j00000gn/T/ipykernel_10266/1738449204.py:2: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(sqlite3.version)  # Should print the SQLite version


In [ ]:
import sqlite3

conn = sqlite3.connect("todolist.db")
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, name TEXT)")
conn.commit()
conn.close()


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("todolist.db")
cursor = conn.cursor()

# Create the conversations table
cursor.execute("""
CREATE TABLE IF NOT EXISTS conversations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    user_message TEXT NOT NULL,
    assistant_message TEXT NOT NULL
)
""")

# Create the memory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS memory (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    key TEXT NOT NULL,
    value TEXT NOT NULL
)
""")

conn.commit()
conn.close()

print("Tables created successfully.")


Tables created successfully.


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("todolist.db")
cursor = conn.cursor()

# Insert a conversation
cursor.execute("""
INSERT INTO conversations (user_message, assistant_message)
VALUES (?, ?)
""", ("Hello, how are you?", "I am fine, thank you!"))

# Insert memory key-value pair
cursor.execute("""
INSERT INTO memory (key, value)
VALUES (?, ?)
""", ("focus", "Saving conversations and memory"))

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Data inserted successfully.")


Data inserted successfully.


In [ ]:
import json
import sqlite3
import logging
import uuid
import openai


# Disable httpx logging
logging.basicConfig(level=logging.CRITICAL)
logging.getLogger("httpx").setLevel(logging.CRITICAL)

# Initialize agent name
agent_name = "Letta"

# Memory structure for storing context
agent_memory = {
    'user_id': None,
    'agent_name': agent_name,
    'user_name': None,
    'previous_answers': {},
    'tasks': {},
    'conversations': []
}

# Database connection
conn = sqlite3.connect("todolist.db")
cursor = conn.cursor()

# Create necessary tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS conversations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    user_message TEXT NOT NULL,
    assistant_message TEXT NOT NULL,
    user_id TEXT NOT NULL
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS memory (
    key TEXT PRIMARY KEY,
    value TEXT NOT NULL,
    user_id TEXT NOT NULL
)
""")
conn.commit()

# Function to save updated memory to the database
def core_memory_save(**kwargs):
    global agent_memory
    for key, value in kwargs.items():
        agent_memory[key] = value

        if isinstance(value, list):
            value = json.dumps(value)

        # Save key-value pairs into the memory table
        cursor.execute("""
        INSERT INTO memory (key, value, user_id)
        VALUES (?, ?, ?)
        ON CONFLICT(key) DO UPDATE SET value = excluded.value
        """, (key, value, agent_memory['user_id']))
    conn.commit()

# Save conversation history into the database
def save_conversation_to_db(user_message, assistant_message):
    cursor.execute("""
    INSERT INTO conversations (user_message, assistant_message, user_id)
    VALUES (?, ?, ?)
    """, (user_message, assistant_message, agent_memory['user_id']))
    conn.commit()

# Define system prompt
def get_system_prompt():
    return f"You are {agent_name}, a friendly assistant who helps users. Use their name if known."

# Check if a user ID already exists in the database
def check_user_id_exists(user_id):
    cursor.execute("""
    SELECT COUNT(*) FROM conversations WHERE user_id = ?
    """, (user_id,))
    return cursor.fetchone()[0] > 0

# Get OpenAI response based on user input
def get_openai_response(user_message):
    try:
        messages = [
            {"role": "system", "content": get_system_prompt()},
            {"role": "user", "content": user_message}
        ]
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main agent step function
def agent_step(user_message):
    global agent_memory

    if agent_memory['user_id'] is None:
        user_input_id = input("Please provide your unique user ID: ") or str(uuid.uuid4())
        if check_user_id_exists(user_input_id):
            agent_memory['user_id'] = user_input_id
            return f"Welcome back, {user_input_id}! You are now logged in."
        else:
            user_input_pw = input("Please set your password: ")
            agent_memory['user_id'] = user_input_id
            core_memory_save(user_id=user_input_id)
            return f"User ID created. Welcome, {user_input_id}!"

    if "my name is" in user_message.lower():
        new_name = user_message.lower().replace("my name is", "").strip()
        agent_memory['user_name'] = new_name
        core_memory_save(user_name=new_name)
        return f"Got it, {new_name}! I'll call you by that name from now on."

    response = get_openai_response(user_message)
    agent_memory['conversations'].append({"role": "user", "content": user_message})
    agent_memory['conversations'].append({"role": "agent", "content": response})
    core_memory_save(conversations=agent_memory['conversations'])
    save_conversation_to_db(user_message, response)
    return response

# Example usage
try:
    print(f"Hello! I am {agent_name}, your assistant.")
    user_input = input("You: ")
    while user_input.lower() != "bye":
        response = agent_step(user_input)
        print(f"{agent_memory['agent_name']}: {response}")
        user_input = input("You: ")
    print(f"{agent_memory['agent_name']}: Goodbye!")
finally:
    conn.close()


Hello! I am Letta, your assistant.
Letta: Welcome back, malvika24! You are now logged in.
Letta: An error occurred: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Letta: An error occurred: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide 